<a href="https://colab.research.google.com/github/tjdrhks0808/Project-Road_Accident/blob/main/Daejeon_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------
# 데이터 정제
---------------------

## full_data 로드

In [ ]:
from geoband.API import *

import math
from tqdm import tqdm
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
import geopy.distance

import pydeck as pdk
import shapely
import shapefile
import os
import re
import mapboxgl
from mapboxgl.viz import *
import seaborn as sns

import platform
import requests; from urllib.parse import urlparse

import warnings
warnings.filterwarnings(action='ignore') #경고 메시지 무시
from IPython.display import display #print가 아닌 display()로 연속 출력

In [ ]:
full_data = gpd.read_file('full_data.geojson')

## 교통량, 혼잡 시간 빈도강도 정리

In [ ]:
#traffic_total_sum , traffic_total_max_sum사용

full_data = full_data.drop(['traffic_small_sum_inter', 'traffic_bus_sum_inter',
                            'traffic_big_sum_inter'], axis = 1)

In [ ]:
full_data.columns

Index(['gid', 'acci_cnt', 'dead', 'serious', 'minor', 'injury', 'dead_ctc',
       'serious_ctc', 'minor_ctc', 'injury_ctc', 'dead_cth', 'serious_cth',
       'minor_cth', 'injury_cth', 'dead_cal', 'serious_cal', 'minor_cal',
       'injury_cal', 'car_car', 'car_human', 'car_only', 'attack_under_10 ',
       'attack_10', 'attack_20', 'attack_30', 'attack_40', 'attack_50',
       'attack_60', 'attack_70', 'attack_80', 'attack_90', 'attack_age_noname',
       'attack_age_unclassified', 'victim_under_10', 'victim_10', 'victim_20',
       'victim_30', 'victim_40', 'victim_50', 'victim_60', 'victim_70',
       'victim_80', 'victim_90', 'victim_age_noname',
       'victim_age_unclassified', 'traffic_total_sum_inter', 'oneway',
       'car_lane', 'barrier_road', 'speed_0', 'speed_30', 'speed_40',
       'speed_50', 'speed_60', 'speed_70', 'speed_80', 'speed_90', 'speed_100',
       'speed_110', 'express', 'general', 'width_1', 'width_2', 'width_3',
       'width_4', 'width_5', 'freq_max_inter

## 속도 변수 정리

In [ ]:
full_data['speed_50'].unique()

array([ 0.,  4.,  2.,  6.,  1.,  8., 17.,  5.,  3., 12.,  7., 10.,  9.,
       15., 11., 18., 13., 16., 14.])

In [ ]:
# 0/30/40~50/60~70/80~110

full_data['speed_40_to_50'] = full_data['speed_40'] + full_data['speed_50']
full_data['speed_60_to_70'] = full_data['speed_60'] + full_data['speed_70']
full_data['speed_80_to_110'] = full_data['speed_80'] + full_data['speed_90']+ full_data['speed_100']+ full_data['speed_110']

In [ ]:
full_data = full_data.drop(['speed_40', 'speed_50', 'speed_60', 'speed_70', 'speed_80', 'speed_90', 'speed_100', 'speed_110'], axis = 1)

In [ ]:
full_data.columns

Index(['gid', 'acci_cnt', 'dead', 'serious', 'minor', 'injury', 'dead_ctc',
       'serious_ctc', 'minor_ctc', 'injury_ctc', 'dead_cth', 'serious_cth',
       'minor_cth', 'injury_cth', 'dead_cal', 'serious_cal', 'minor_cal',
       'injury_cal', 'car_car', 'car_human', 'car_only', 'attack_under_10 ',
       'attack_10', 'attack_20', 'attack_30', 'attack_40', 'attack_50',
       'attack_60', 'attack_70', 'attack_80', 'attack_90', 'attack_age_noname',
       'attack_age_unclassified', 'victim_under_10', 'victim_10', 'victim_20',
       'victim_30', 'victim_40', 'victim_50', 'victim_60', 'victim_70',
       'victim_80', 'victim_90', 'victim_age_noname',
       'victim_age_unclassified', 'traffic_total_sum_inter', 'oneway',
       'car_lane', 'barrier_road', 'speed_0', 'speed_30', 'express', 'general',
       'width_1', 'width_2', 'width_3', 'width_4', 'width_5', 'freq_max_inter',
       'time_max_inter', 'intersects', 'TURN_P', 'building_tot', 'house',
       'office', 'other', 'crosswal

## 연령대 정리

In [ ]:
full_data['youth_attack'] = full_data['attack_20'] + full_data['attack_30']
full_data['senior_attack'] = full_data['attack_40'] + full_data['attack_50'] + full_data['attack_60']
full_data['weak_attack'] = full_data['attack_under_10 '] + full_data['attack_10'] + full_data['attack_70']+ full_data['attack_80']+ full_data['attack_90']
full_data['attack_na'] = full_data['attack_age_noname']+full_data['attack_age_unclassified']

In [ ]:
full_data['youth_victim'] = full_data['victim_20'] + full_data['victim_30']
full_data['senior_victim'] = full_data['victim_40'] + full_data['victim_50'] + full_data['victim_60']
full_data['weak_victim'] = full_data['victim_under_10'] + full_data['victim_10'] + full_data['victim_70']+ full_data['victim_80']+ full_data['victim_90']
full_data['victim_na'] = full_data['victim_age_noname']+full_data['victim_age_unclassified']

In [ ]:
full_data = full_data.drop(['attack_under_10 ', 'attack_10', 'attack_20',
       'attack_30', 'attack_40', 'attack_50', 'attack_60', 'attack_70',
       'attack_80', 'attack_90', 'attack_age_noname',
       'attack_age_unclassified'], axis =1)

In [ ]:
full_data = full_data.drop(['victim_under_10', 'victim_10', 'victim_20',
       'victim_30', 'victim_40', 'victim_50', 'victim_60', 'victim_70',
       'victim_80', 'victim_90', 'victim_age_noname',
       'victim_age_unclassified'], axis=1)

In [ ]:
full_data = full_data.drop(['crosswalk_cnt'], axis=1)

In [ ]:
full_data.columns

Index(['gid', 'acci_cnt', 'dead', 'serious', 'minor', 'injury', 'dead_ctc',
       'serious_ctc', 'minor_ctc', 'injury_ctc', 'dead_cth', 'serious_cth',
       'minor_cth', 'injury_cth', 'dead_cal', 'serious_cal', 'minor_cal',
       'injury_cal', 'car_car', 'car_human', 'car_only',
       'traffic_total_sum_inter', 'oneway', 'car_lane', 'barrier_road',
       'speed_0', 'speed_30', 'express', 'general', 'width_1', 'width_2',
       'width_3', 'width_4', 'width_5', 'freq_max_inter', 'time_max_inter',
       'intersects', 'TURN_P', 'building_tot', 'house', 'office', 'other',
       'crosswalk_ar', 'noparking_ar', 'trlight_pas', 'trlight_car', 'cctv',
       'sign_traffic', 'safe_zone', 'pop_all', 'pop_sen', 'pop_prd', 'pop_chd',
       'cars_cnt', 'subway_cnt', 'parkinglot_cnt', 'school_cnt', 'ssm_cnt',
       'restaurant_cnt', 'geometry', 'speed_40_to_50', 'speed_60_to_70',
       'speed_80_to_110', 'youth_attack', 'senior_attack', 'weak_attack',
       'attack_na', 'youth_victim', 'sen

In [ ]:
full_data

,gid,acci_cnt,dead,serious,minor,injury,dead_ctc,serious_ctc,minor_ctc,injury_ctc,...,speed_60_to_70,speed_80_to_110,youth_attack,senior_attack,weak_attack,attack_na,youth_victim,senior_victim,weak_victim,victim_na
0,다바866110,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,다바823157,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,다바928260,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,다바931203,2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
4,다바836109,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54907,다바861065,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54908,다바912036,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54909,다바867189,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54910,다바851062,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 파생변수 정리

### 파생변수1. isroad : 도로 정보가 할당되었는지

In [ ]:
full_data['isroad'] = (full_data.speed_0 + full_data.speed_30 + full_data.speed_40_to_50 +
                       full_data.speed_60_to_70 + full_data.speed_80_to_110)

full_data.isroad = [1 if x>0 else 0 for x in full_data.isroad]

### 파생변수2. acci_risk : EPDO.ver2 적용한 사고위험도

In [ ]:
full_data['acci_risk'] = (full_data.dead * 12) + (full_data.serious * 3) + full_data.minor + full_data.injury

full_data['acci_risk_ctc'] = (full_data.dead_ctc * 12) + (full_data.serious_ctc * 3) + full_data.minor_ctc + full_data.injury_ctc
full_data['acci_risk_cth'] = (full_data.dead_cth * 12) + (full_data.serious_cth * 3) + full_data.minor_cth + full_data.injury_cth
full_data['acci_risk_cal'] = (full_data.dead_cal * 12) + (full_data.serious_cal * 3) + full_data.minor_cal + full_data.injury_cal

### 파생변수3. acci_ratio : acci_cnt/traffic_total_sum_inter

In [ ]:
full_data['acci_ratio'] = full_data.acci_cnt / np.array([ (x+1) if x == 0 else x for x in full_data.traffic_total_sum_inter ])

full_data['acci_ratio_ctc'] = full_data.car_car / np.array([ (x+1) if x == 0 else x for x in full_data.traffic_total_sum_inter ])
full_data['acci_ratio_cth'] = full_data.car_human / np.array([ (x+1) if x == 0 else x for x in full_data.traffic_total_sum_inter ])
full_data['acci_ratio_cal'] = full_data.car_only / np.array([ (x+1) if x == 0 else x for x in full_data.traffic_total_sum_inter ])

### 종속변수 정리

In [ ]:
full_data = full_data.drop(['dead', 'serious', 'minor', 'injury',
                            'dead_ctc', 'serious_ctc', 'minor_ctc', 'injury_ctc',
                            'dead_cth', 'serious_cth', 'minor_cth', 'injury_cth',
                            'dead_cal', 'serious_cal', 'minor_cal', 'injury_cal',
                            ], axis=1)

## 정리된 데이터 저장 : daejeon_data

In [ ]:
full_data.to_file('daejeon_data.geojson', driver='GeoJSON')